# Fake News Classifier using Bidirectional LSTM

In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('/Users/aravindmohan/Desktop/TFL/Datasets/fake_news_train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df=df.dropna()

In [7]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
#Get independent features
X=df.drop('label',axis=1)
y=df['label']

In [12]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.3.1'

In [45]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [17]:
#vocabulary size
# Considering 5000 words in dictionary 
voc_size=5000

In [18]:
messages=X.copy()

In [24]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [25]:
#stopwords.words('english')

In [27]:
messages.reset_index(inplace=True)

In [38]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [39]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [40]:
onehot_rep=[one_hot(words,voc_size) for words in corpus]
onehot_rep

[[1759, 810, 2525, 2120, 4703, 4017, 1926, 4075, 1656, 4314],
 [910, 1936, 1896, 1072, 3353, 157, 1389],
 [4416, 774, 2105, 4018],
 [3911, 1234, 3991, 88, 3368, 2154],
 [3400, 3353, 2640, 3646, 415, 1906, 3353, 2049, 2110, 1841],
 [251,
  3948,
  4347,
  1173,
  4071,
  4447,
  3710,
  246,
  171,
  1300,
  4441,
  2388,
  291,
  502,
  1389],
 [4263, 4143, 3844, 455, 4415, 4650, 4982, 2512, 4589, 1595, 4938],
 [4761, 3690, 1392, 3206, 4779, 2926, 4447, 2904, 4589, 1595, 4938],
 [1631, 1428, 277, 3486, 3189, 4442, 4549, 4686, 4447, 3154],
 [2400, 3519, 4941, 3733, 3201, 2747, 132, 2399],
 [1371, 3569, 4009, 3148, 1245, 3082, 2519, 676, 3099, 2916, 3933],
 [88, 1256, 4703, 4442, 4447, 4779],
 [4064, 1781, 4423, 1805, 3977, 3054, 2881, 2594, 3562],
 [778, 4768, 2911, 4252, 3222, 3606, 2906, 4589, 1595, 4938],
 [4713, 2163, 622, 1546, 1402, 4589, 1595, 4938],
 [1420, 625, 1845, 320, 1630, 1711, 2760, 1923, 2874, 4268],
 [2202, 1174, 1936],
 [4546, 3948, 1514, 2206, 4447, 2389, 4943, 1389]

In [42]:
#Embedding Representation

In [43]:
sent_length=20
embedded_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 4075, 1656, 4314],
       [   0,    0,    0, ..., 3353,  157, 1389],
       [   0,    0,    0, ...,  774, 2105, 4018],
       ...,
       [   0,    0,    0, ..., 4589, 1595, 4938],
       [   0,    0,    0, ...,   17, 3625, 1423],
       [   0,    0,    0, ..., 4636, 1323, 3504]], dtype=int32)

In [46]:
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary()) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [47]:
len(embedded_docs),y.shape 

(18285, (18285,))

In [48]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [65]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 5s 12ms/step - loss: 0.2815 - accuracy: 0.8731 - val_loss: 0.1911 - val_accuracy: 0.9183
Epoch 2/10
383/383 [==============================] - 4s 12ms/step - loss: 0.1373 - accuracy: 0.9465 - val_loss: 0.2028 - val_accuracy: 0.9231
Epoch 3/10
383/383 [==============================] - 4s 12ms/step - loss: 0.0980 - accuracy: 0.9642 - val_loss: 0.2027 - val_accuracy: 0.9206
Epoch 4/10
383/383 [==============================] - 4s 12ms/step - loss: 0.0703 - accuracy: 0.9762 - val_loss: 0.2293 - val_accuracy: 0.9208
Epoch 5/10
383/383 [==============================] - 5s 14ms/step - loss: 0.0491 - accuracy: 0.9824 - val_loss: 0.2617 - val_accuracy: 0.9198
Epoch 6/10
383/383 [==============================] - 5s 13ms/step - loss: 0.0397 - accuracy: 0.9869 - val_loss: 0.3006 - val_accuracy: 0.9167
Epoch 7/10
383/383 [==============================] - 4s 11ms/step - loss: 0.0294 - accuracy: 0.9910 - val_loss: 0.3274 - val_accuracy: 0.9125

In [66]:
y_pred=model.predict(X_test)

In [67]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [68]:
from sklearn.metrics import confusion_matrix

In [69]:
confusion_matrix(y_test,y_pred)

array([[3123,  296],
       [ 239, 2377]])

In [72]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9113504556752279

In [73]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

